In [44]:
#Importamos librerias y leemos los archivos de datos. 

import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn import tree
from sklearn.metrics import f1_score
import lightgbm as lgb

train_values = pd.read_csv('train_values.csv', index_col='building_id')
train_labels = pd.read_csv('train_labels.csv', index_col='building_id')

test_values = pd.read_csv('test_values.csv', index_col='building_id')
submission_format = pd.read_csv('submission_format.csv', index_col='building_id')

train_values = pd.get_dummies(train_values)
test_values = pd.get_dummies(test_values)

In [45]:
lgb_train = lgb.Dataset(train_values, train_labels)
lgb_eval = lgb.Dataset(train_values, train_labels, reference=lgb_train)

params = {
    'objective': 'regression',
}

gbm = lgb.train(params,
                lgb_train,
                num_boost_round=100,
                valid_sets=lgb_eval,
                early_stopping_rounds=10)

predicciones_lightgbm = gbm.predict(train_values, num_iteration=gbm.best_iteration)

predicciones_lightgbm=[int(x+0.5) for x in predicciones_lightgbm]

f1_score(train_labels, predicciones_lightgbm, average='micro')

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.024673 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 817
[LightGBM] [Info] Number of data points in the train set: 260601, number of used features: 68
[LightGBM] [Info] Start training from score 2.238272
[1]	valid_0's l2: 0.35129
Training until validation scores don't improve for 10 rounds
[2]	valid_0's l2: 0.332626
[3]	valid_0's l2: 0.317458
[4]	valid_0's l2: 0.305029
[5]	valid_0's l2: 0.294809
[6]	valid_0's l2: 0.28644
[7]	valid_0's l2: 0.279538
[8]	valid_0's l2: 0.273715
[9]	valid_0's l2: 0.268834
[10]	valid_0's l2: 0.26479
[11]	valid_0's l2: 0.261324
[12]	valid_0's l2: 0.258593
[13]	valid_0's l2: 0.256082
[14]	valid_0's l2: 0.254043
[15]	valid_0's l2: 0.251878
[16]	valid_0's l2: 0.250142
[17]	valid_0's l2: 0.248852
[18]	valid_0's l2: 0.247551
[19]	valid_0's l2: 0.246569
[20]	valid

0.7055997482741816

In [46]:
predicciones_lightgbm = gbm.predict(test_values, num_iteration=gbm.best_iteration)

predicciones_lightgbm =[int(x+0.5) for x in predicciones_lightgbm]

In [47]:
my_submission = pd.DataFrame(data=predicciones_lightgbm,
                             columns=submission_format.columns,
                             index=submission_format.index)

my_submission.damage_grade = my_submission.damage_grade.round()

my_submission.damage_grade = my_submission.damage_grade.astype(int)

my_submission.to_csv('submission_lightgbm.csv')